In [ ]:
def find_best_list(list, fidelity_function):
  """
  리스트의 fidelity를 99.99% 이상으로 높이는 최적의 리스트를 찾는 함수입니다.

  Args:
    list: fidelity를 계산할 리스트입니다.
    fidelity_function: fidelity를 계산하는 함수입니다.

  Returns:
    fidelity가 99.99% 이상인 최적의 리스트입니다.
  """

  # 리스트의 요소를 순서대로 탐색합니다.
  for i in range(len(list)):
    # 현재 요소를 0, 1, 2, 4 중 하나로 바꿔서 fidelity를 계산합니다.
    for j in range(5):
      new_list = list[:i] + [j] + list[i + 1:]
      fidelity = fidelity_function(new_list)

      # 현재 리스트보다 fidelity가 더 높으면, 최종 리스트로 선택합니다.
      if fidelity > fidelity_function(list):
        list = new_list

  return list


def fidelity_function(list):
  """
  리스트의 fidelity를 계산하는 함수입니다.

  Args:   
    list: fidelity를 계산할 리스트입니다.

  Returns:
    리스트의 fidelity입니다.
  """

  # TODO: fidelity를 계산하는 실제 함수를 구현하세요.
  return 0.0


if __name__ == "__main__":
  # 예시 리스트
  list = [4, 1, 3, 2, 4, 4, 1, 2, 2, 2, 3, 3]

  # fidelity 함수
  fidelity_function = lambda list: sum(list) / len(list)

  # 최적의 리스트를 찾습니다.
  best_list = find_best_list(list, fidelity_function)

  # 결과를 출력합니다.
  print(best_list)
